In [26]:

import pandas
import binance
from binance.client import Client
from binance import Client
import time
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
import tensorflow as tf

from keys import get_api_key, get_secret_key

api_key = get_api_key()
api_secret = get_secret_key()

class Data(StandardScaler):
    def transform_data(self, close_price, close_time):
        self.price = np.array([prices[i] for i in range(len(prices))])
        self.close_time = np.array([datetime.fromtimestamp(time[i] / 1000).strftime("%H:%M:%S") for i in range(len(time))])
        self.price = self.price.reshape(self.price.shape[0], 1)

        self.scaler = StandardScaler()
        self.scaler.fit(self.price[:375])
        self.price = self.scaler.transform(self.price)
        self.price = self.price.reshape(int(self.price.shape[0]/5), 5)
        return self.price, self.close_time, self.scaler

    def prepare_data(self, data):
        self.x_train = df.iloc[:75, :4]
        self.x_test = df.iloc[75:, :4]

        self.y_train = df.iloc[:75, -1:]
        self.y_test = df.iloc[75:, -1:]

        self.x_train = np.array(self.x_train)
        self.x_test = np.array(self.x_test)

        self.y_train = np.array(self.y_train)
        self.y_test = np.array(self.y_test)

        self.x_train = self.x_train.reshape(self.x_train.shape[0], self.x_train.shape[1], 1)
        self.x_test = self.x_test.reshape(self.x_test.shape[0], self.x_test.shape[1], 1)
        return self.x_train, self.x_test, self.y_train, self.y_test

class Bot(binance.Client, Data):
    def __init__(self):
        super().__init__()
        self.client = Client(api_key, api_secret)

    def get_price_1min(self, ticker):
        self.price = self.client.get_klines(symbol=ticker, interval=Client.KLINE_INTERVAL_1MINUTE)
        return self.price

    def network(self, x, y):
        self.model = Sequential()
        self.model.add(LSTM(20, return_sequences=True, input_shape=(self.x_train.shape[1:])))
        self.model.add(BatchNormalization())
        self.model.add(LSTM(40, return_sequences=True, input_shape=(self.x_train.shape[1:])))
        self.model.add(BatchNormalization())
        self.model.add(LSTM(60, return_sequences=True, input_shape=(self.x_train.shape[1:])))
        self.model.add(BatchNormalization())
        self.model.add(LSTM(80, return_sequences=False, input_shape=(self.x_train.shape[1:])))
        self.model.add(Dense(1, activation="linear"))
        self.model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
        return self.model


if __name__ == "__main__":
    bot = Bot()

tick = input("ticker: ")
orders = bot.get_price_1min(tick)

df = pandas.DataFrame(bot.price, columns=["Open time", "Open", "High", "Low", "Close", "Volume", "Close time",
                                        "Quote asset volume", "Number of trades", "Taker buy base asset volume",
                                        "Taker buy quote asset volume", "Can be ignored"])
prices = df["Close"]
time = df["Close time"]
transform = bot.transform_data(prices, time)
df = pd.DataFrame(bot.price, columns=["First", "Second", "Third", "Fourth", "Target"])

prepare = bot.prepare_data(df)
nn = bot.network(bot.x_train, bot.y_train)

bot.model.fit(bot.x_train, bot.y_train, epochs=1000, batch_size=5)

y_pred = bot.model.predict(bot.x_test)
x = bot.scaler.inverse_transform(y_pred)
test = bot.scaler.inverse_transform(bot.x_test)

for e in x:
    print(e)


Epoch 1/10
15/15 [==============================] - 0s 2ms/step - loss: 0.8871 - accuracy: 0.0000e+00
Epoch 2/10
15/15 [==============================] - 0s 2ms/step - loss: 0.5342 - accuracy: 0.0000e+00
Epoch 3/10
15/15 [==============================] - 0s 2ms/step - loss: 0.1608 - accuracy: 0.0000e+00
Epoch 4/10
15/15 [==============================] - 0s 2ms/step - loss: 0.0562 - accuracy: 0.0000e+00
Epoch 5/10
15/15 [==============================] - 0s 2ms/step - loss: 0.0467 - accuracy: 0.0000e+00
Epoch 6/10
15/15 [==============================] - 0s 2ms/step - loss: 0.0451 - accuracy: 0.0000e+00
Epoch 7/10
15/15 [==============================] - 0s 2ms/step - loss: 0.0438 - accuracy: 0.0000e+00
Epoch 8/10
15/15 [==============================] - 0s 2ms/step - loss: 0.0431 - accuracy: 0.0000e+00
Epoch 9/10
15/15 [==============================] - 0s 2ms/step - loss: 0.0429 - accuracy: 0.0000e+00
Epoch 10/10
15/15 [==============================] - 0s 2ms/step - loss: 0.0422 - 